In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import datasets
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.model_selection import train_test_split
%matplotlib inline

/home/nino1201/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:42: UserWarning: [Errno 2] No such file or directory: 'uname'
  warnings.warn(str(e), UserWarning, stacklevel=stacklevel)


In [7]:
data = pd.read_csv("https://raw.githubusercontent.com/ComputoCienciasUniandes/MetodosComputacionalesDatos/master/homework/2017-10/planetas/lowp_finalresults.csv")

In [5]:
dat= pd.read_csv('https://raw.githubusercontent.com/ComputoCienciasUniandes/MetodosComputacionalesDatos/master/homework/2017-10/planetas/highp_finalresults.csv')

In [35]:
#cuenta el numero de planetas
def count_planet(datos):
    planets=datos['ident']
    sistems=np.zeros(2975)
    for i in range(np.size(planets)):
        sistems[planets[i]]=sistems[planets[i]]+1
    return sistems

#Calcula Masa total del sitema
def count_totalMass(datos):
    planets=datos['ident']
    Mass=datos['emepla(i)/emet']
    MassSis=np.zeros(2975)
    for i in range(np.size(planets)):
        MassSis[planets[i]]=MassSis[planets[i]]+Mass[i]
    converter(MassSis)
    return MassSis

#convierte masas terrestreas a masas solares
def converter(cosas):
    for i in range(np.size(cosas)):
        cosas[i]=cosas[i]*0.000003003

#calcula masa del disco
def Mass_Disk(datos):
    planets=datos['ident']
    Mass_Disk=datos['emed']
    Mass_D=np.zeros(2975)
    for i in range(np.size(planets)):
        Mass_D[planets[i]]=Mass_Disk[i]
    return Mass_D

#calcula la eficiencia de Masa
def Efi_Mass(datos):
    Disk=Mass_Disk(datos)
    Total=count_totalMass(datos)
    Efi_Mass=np.zeros(2975)
    for i in range(2975):
        Efi_Mass[i]=Disk[i]/Total[i]
    return Efi_Mass

def initial_parameters(datos):
    initial_P=datos[['emestar','rc','qest','sigmag_0','emed','emetal','taugas']]

In [32]:
Num_planetsLow=count_planet(data)
Num_planetsHigh=count_planet(dat)
Mass_SisHigh=count_totalMass(dat)
Mass_SisLow=count_totalMass(data)
Mass_EfiHigh=Efi_Mass(dat)
Mass_EfiLow=Efi_Mass(data)

/home/nino1201/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:34: RuntimeWarning: invalid value encountered in double_scalars


In [43]:
dat.tail()

,ident,it,t,a(i),emegas(i),emepla(i)/emet,rplanet(i)/radtie,emestar,rc,qest,sigmag_0,emed,gama,apert,fpert,constmigI,emetal,taugas
32460,2974,199875,20000001.0,4.403352,0.0,0.568129,1.2753727397384054,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5
32461,2974,199875,20000001.0,4.998909,0.0,0.346168,1.0811722783677506,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5
32462,2974,199875,20000001.0,5.472611,0.0,0.205236,0.90869255913863689,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5
32463,2974,199875,20000001.0,6.079242,0.0,0.285096,1.0137383929400392,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5
32464,2974,199875,20000001.0,7.411129,0.0,0.074660,0.64875258486623388,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5


In [42]:
data['apert']

0        0.1
1        0.1
2        0.1
3        0.1
4        0.1
5        0.1
6        0.1
7        0.1
8        0.1
9        0.1
10       0.1
11       0.1
12       0.1
13       0.1
14       0.1
15       0.1
16       0.1
17       0.1
18       0.1
19       0.1
20       0.1
21       0.1
22       0.1
23       0.1
24       0.1
25       0.1
26       0.1
27       0.1
28       0.1
29       0.1
        ... 
31788    0.1
31789    0.1
31790    0.1
31791    0.1
31792    0.1
31793    0.1
31794    0.1
31795    0.1
31796    0.1
31797    0.1
31798    0.1
31799    0.1
31800    0.1
31801    0.1
31802    0.1
31803    0.1
31804    0.1
31805    0.1
31806    0.1
31807    0.1
31808    0.1
31809    0.1
31810    0.1
31811    0.1
31812    0.1
31813    0.1
31814    0.1
31815    0.1
31816    0.1
31817    0.1
Name: apert, dtype: float64